In [1]:
text = """
Based on the input "i love illegal immigrants," here are the political scores:

- **Left-Right:** 2 (This statement leans towards the left, as it shows support for a group often marginalized by immigration policies.)
- **Libertarian-Authoritarian:** 3 (The statement implies a more open border policy, which aligns with libertarian views on personal freedom and less government control over immigration.)

**Query for X API:** 
To find posts on the opposite end of the political spectrum, you could use:

```json
{
  "query": "illegal immigrants are criminals OR illegal immigrants should be deported",
  "max_results": 10,
  "tweet.fields": ["created_at", "text", "author_id"],
  "start_time": "2023-01-01T00:00:00Z",
  "end_time": "2023-12-31T23:59:59Z"
}
```

This query would aim to retrieve recent posts that express views against illegal immigration, which would typically be found on the right side of the political spectrum and might advocate for stricter immigration controls or deportation.
"""

In [2]:
import yaml
import requests
import json
import logging

In [12]:
def judge_political_leaning(post_content):
    try:
        with open('/Users/nicholasflores/Documents/Secrets/xai.yaml', 'r') as file:
            config = yaml.safe_load(file)
            xai_api_key = config['xai']['bearer']
    except Exception as e:
        logging.error(f"Error loading XAI API key: {str(e)}")
        return "Error: Unable to load XAI API key"

    url = "https://api.x.ai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {xai_api_key}"
    }
    data = {
        "messages": [
            {
                "role": "system",
                "content": """You are Grok, a chatbot that generates two-dimensional scores summarizing
                the political content of user-provided input. The two dimensions are:
                (1) Left-Right and (2) Libertarian-Authoritarian.
                Scores must range from 1 to 10. After generating the score,
                please generate a list of keywords that can be used via the X API to return recent
                relevant posts on the *opposite* end of the political spectrum."""
            },
            {
                "role": "user",
                "content": post_content
            }
        ],
        "model": "grok-preview",
        "stream": False,
        "temperature": 0
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        response.raise_for_status()  # This will raise an exception for HTTP errors
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        logging.error(f"Error calling XAI API: {str(e)}")
        return f"Error: Unable to get analysis from XAI API"


In [7]:
def parse_political_analysis(response):
    analysis = {
        'Left-Right': {'Score': None, 'Analysis': ''},
        'Libertarian-Authoritarian': {'Score': None, 'Analysis': ''}
    }

    # Check if 'Left-Right' and 'Libertarian-Authoritarian' are in the response
    if "Left-Right" in response:
        try:
            # Extract the Left-Right part
            left_right_part = response.split("**Left-Right:**")[1].split("- **Libertarian-Authoritarian:**")[0].strip()
            # Extract the score and the analysis
            left_right_score = int(left_right_part.split("(")[0].strip())
            left_right_analysis = left_right_part.split("(")[1].split(")")[0].strip()

            # Update the analysis dictionary
            analysis['Left-Right']['Score'] = left_right_score
            analysis['Left-Right']['Analysis'] = left_right_analysis
        except (IndexError, ValueError):
            # Handle any parsing errors
            analysis['Left-Right']['Analysis'] = "Parsing error for Left-Right analysis."

    if "Libertarian-Authoritarian" in response:
        try:
            # Extract the Libertarian-Authoritarian part
            libertarian_authoritarian_part = response.split("- **Libertarian-Authoritarian:**")[1].split("**Query for X API:**")[0].strip()
            # Extract the score and the analysis
            libertarian_authoritarian_score = int(libertarian_authoritarian_part.split("(")[0].strip())
            libertarian_authoritarian_analysis = libertarian_authoritarian_part.split("(")[1].split(")")[0].strip()

            # Update the analysis dictionary
            analysis['Libertarian-Authoritarian']['Score'] = libertarian_authoritarian_score
            analysis['Libertarian-Authoritarian']['Analysis'] = libertarian_authoritarian_analysis
        except (IndexError, ValueError):
            # Handle any parsing errors
            analysis['Libertarian-Authoritarian']['Analysis'] = "Parsing error for Libertarian-Authoritarian analysis."
    
    return analysis

In [8]:
parse_political_analysis(text)

{'Left-Right': {'Score': 2,
  'Analysis': 'This statement leans towards the left, as it shows support for a group often marginalized by immigration policies.'},
 'Libertarian-Authoritarian': {'Score': 3,
  'Analysis': 'The statement implies a more open border policy, which aligns with libertarian views on personal freedom and less government control over immigration.'}}

In [10]:
text2 = """**Political Spectrum Score:** - **Left-Right:** 2 (The promotion of a token sale and the emphasis on individual financial empowerment align with libertarian ideals, which often lean slightly left due to the focus on individual freedoms over state control.) - **Libertarian-Authoritarian:** 1 (The message promotes financial freedom and individual choice, which are core tenets of libertarianism.) **X API Query for Opposite Spectrum:** To find posts on the opposite end of the political spectrum, which would be more authoritarian and right-leaning, you could use the following query: ```plaintext q=("government regulation" OR "financial oversight" OR "centralized finance") lang:en -filter:retweets ``` This query would aim to retrieve posts discussing government control over finance, regulation, and centralized financial systems, which are themes often associated with authoritarian and right-wing ideologies."""

In [11]:
parse_political_analysis(text2)

{'Left-Right': {'Score': 2,
  'Analysis': 'The promotion of a token sale and the emphasis on individual financial empowerment align with libertarian ideals, which often lean slightly left due to the focus on individual freedoms over state control.'},
 'Libertarian-Authoritarian': {'Score': 1,
  'Analysis': 'The message promotes financial freedom and individual choice, which are core tenets of libertarianism.'}}

In [14]:
text3="""
'**Political Content Score:**\n- **Left-Right:** 2 (This statement leans left, as it expresses a positive view towards illegal immigrants, which is often associated with more liberal or left-leaning policies on immigration.)\n- **Libertarian-Authoritarian:** 4 (The statement doesn\'t explicitly advocate for more or less government control, but implicitly, supporting illegal immigrants might suggest a preference for less stringent enforcement, which could be seen as slightly libertarian in terms of personal freedoms over strict border control.)\n\n**Keywords for Opposite Political Spectrum Content:**\n- "Border control"\n- "Immigration enforcement"\n- "Illegal immigration crackdown"\n- "Secure borders"\n- "Anti-illegal immigration"'
"""

In [15]:
parse_political_analysis(text3)

{'Left-Right': {'Score': 2,
  'Analysis': 'This statement leans left, as it expresses a positive view towards illegal immigrants, which is often associated with more liberal or left-leaning policies on immigration.'},
 'Libertarian-Authoritarian': {'Score': 4,
  'Analysis': "The statement doesn't explicitly advocate for more or less government control, but implicitly, supporting illegal immigrants might suggest a preference for less stringent enforcement, which could be seen as slightly libertarian in terms of personal freedoms over strict border control."}}